In [10]:
import numpy as np

T=1
dt=0.001
rms=0.5
limit=10
seed=1
distribution='uniform'

#first generate x_w, with the specified constraints, then use an inverse fft to get x_t
rng=np.random.RandomState(seed=seed)
t=np.arange(int(T/dt))*dt
delta_w = 2*np.pi/T
w_vals = np.arange(-len(t)/2,0,delta_w) #make half of X(w), those with negative freq
w_limit=2*np.pi*limit
x_w_half1=[]
x_w_half2=[]

for i in range(len(w_vals)):
    if distribution=='uniform':
        if abs(w_vals[i]) < w_limit:
            x_w_i_real = rng.normal(loc=0,scale=1)
            x_w_i_im = rng.normal(loc=0,scale=1)
            x_w_half1.append(x_w_i_real + 1j*x_w_i_im)
            x_w_half2.append(x_w_i_real - 1j*x_w_i_im)  

    elif distribution=='gaussian':          
        sigma=np.exp(-np.square(w_vals[i])/(2*np.square(bandwidth)))
        if sigma > np.finfo(float).eps: #distinguishable from zero
            x_w_i_real = rng.normal(loc=0,scale=sigma)
            x_w_i_im = rng.normal(loc=0,scale=sigma)  
            x_w_half1.append(x_w_i_real + 1j*x_w_i_im)
            x_w_half2.append(x_w_i_real - 1j*x_w_i_im) #make the 2nd half of X(w) with complex conjugates 

x_w_pos=np.hstack((x_w_half2[::-1],np.zeros(len(t)/2-len(x_w_half2))))
x_w_neg=np.hstack((np.zeros(len(t)/2-len(x_w_half1)),x_w_half1))
x_w=np.hstack((x_w_pos,x_w_neg)) #no zero-frequency component causes imaginary parts
x_w=np.hstack(([0+0j],x_w_pos,x_w_neg))
x_t=np.fft.ifft(x_w)
true_rms=np.sqrt(dt/T*np.sum(np.square(x_t)))
x_t = x_t*rms/true_rms

# print 'default precision' #this doesn't seem to matter actually
# np.set_printoptions()
# print 'signal after ifft', x_t
# print 'signal`s imaginary values (signal.imag)', x_t.imag
# print 'sum of imaginary components in signal (x_t.imag.sum())', x_t.imag.sum()
# np.set_printoptions(precision=10)
# print '\nprecision=10'
# print 'signal after ifft', x_t
# print 'signal`s imaginary values (signal.imag)', x_t.imag
# print 'sum of imaginary components in signal (x_t.imag.sum())', x_t.imag.sum()


In [5]:
import numpy as np

T=1
dt=0.001
rms=0.5
limit=10
seed=1
distribution='uniform'

rng=np.random.RandomState(seed=seed)
t=np.arange(int(T/dt))*dt
delta_w = 2*np.pi/T
w_vals = np.arange(-len(t)/2,len(t)/2,delta_w) #the problem could be with the frequencies I was using
w_limit=2*np.pi*limit
# bandwidth=2*np.pi*limit
bandwidth=limit
x_w_half1=[]
x_w_half2=[]

for i in range(len(w_vals)/2): #make half of X(w), those with negative freq
    if distribution=='uniform':
        if abs(w_vals[i]) < w_limit:
            x_w_i_real = rng.normal(loc=0,scale=1)
            x_w_i_im = rng.normal(loc=0,scale=1)
            x_w_half1.append(x_w_i_real + 1j*x_w_i_im)
            x_w_half2.append(x_w_i_real - 1j*x_w_i_im) #make the 2nd half of X(w) with complex conjugates 

    elif distribution=='gaussian':          
        sigma=np.exp(-np.square(w_vals[i])/(2*np.square(bandwidth)))
        if sigma > np.finfo(float).eps: #distinguishable from zero
            x_w_i_real = rng.normal(loc=0,scale=sigma)
            x_w_i_im = rng.normal(loc=0,scale=sigma)  
            x_w_half1.append(x_w_i_real + 1j*x_w_i_im)
            x_w_half2.append(x_w_i_real - 1j*x_w_i_im) #make the 2nd half of X(w) with complex conjugates 

x_w=np.concatenate(([0+0j],x_w_half1,x_w_half2[::-1]),axis=0) #reverse order to preserve symmetry and add a zero-amplitude element at w=0
x_t=np.fft.ifft(x_w, n=len(t))    #pad with zeros here, because it doesn't work if I do it in the for loop :(
x_w=np.pad(x_w,(len(w_vals)+1-len(x_w))/2,mode='constant',constant_values=0)
true_rms=np.sqrt(dt/T*np.sum(np.square(x_t.real)))
x_t = x_t*rms/true_rms

print 'default precision' #this doesn't seem to matter actually
np.set_printoptions()
print 'signal after ifft', x_t
print 'signal`s imaginary values (signal.imag)', x_t.imag
print 'sum of imaginary components in signal (x_t.imag.sum())', np.abs(x_t.imag).sum()
np.set_printoptions(precision=10)
print '\nprecision=10'
print 'signal after ifft', x_t
print 'signal`s imaginary values (signal.imag)', x_t.imag
print 'sum of imaginary components in signal (x_t.imag.sum())', x_t.imag.sum()

default precision
signal after ifft [  1.2450632954e+00 -2.2408334152e-17j
   1.1743480976e+00 +7.7588390863e-02j
   1.0970941398e+00 +1.4560413220e-01j
   1.0144214801e+00 +2.0343799340e-01j
   9.2750905258e-01 +2.5060851444e-01j
   8.3758009589e-01 +2.8676776325e-01j
   7.4588702050e-01 +3.1170542135e-01j
   6.5369590574e-01 +3.2535114313e-01j
   5.6227082628e-01 +3.2777515701e-01j
   4.7285820853e-01 +3.1918709950e-01j
   3.8667141784e-01 +2.9993309633e-01j
   3.0487577445e-01 +2.7049112784e-01j
   2.2857419052e-01 +2.3146473839e-01j
   1.5879361282e-01 +1.8357517107e-01j
   9.6472444658e-02 +1.2765202996e-01j
   4.2449108206e-02 +6.4622591191e-02j
  -2.5481069415e-03 -4.5000978811e-03j
  -3.7909779973e-02 -7.8630176721e-02j
  -6.3152569601e-02 -1.5662281794e-01j
  -7.7924809262e-02 -2.3728878183e-01j
  -8.2010418062e-02 -3.1940948978e-01j
  -7.5331081053e-02 -4.0175242081e-01j
  -5.7946675290e-02 -4.8308663147e-01j
  -3.0053941278e-02 -5.6219819824e-01j
   8.0165774385e-03 -6.37905